In [ ]:
from gensim.models import Word2Vec
import pandas as pd
import gensim
import jieba
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer


# 读取训练文本数据，每行代表一个文本数据
train_data = pd.read_csv('./corpus_0.csv', sep='\t')
train_data.fillna('', inplace=True)
train_data_0 = train_data.iloc[1:]

# 定义分词函数
def cut_words(sentence):
    return [word for word in jieba.cut(sentence) if word.strip()]

# 预处理语料库
sentences_1 = [cut_words(str(text)) for text in train_data_0['corpus']]

sentences_as_texts_1 = [' '.join(sentence) for sentence in sentences]

# 读取训练文本数据，每行代表一个文本数据
train_data = pd.read_csv('./corpus_5.csv', sep='\t')
train_data.fillna('', inplace=True)
train_data_5 = train_data.iloc[1:]

# 定义分词函数
def cut_words(sentence):
    return [word for word in jieba.cut(sentence) if word.strip()]

# 预处理语料库
sentences_2 = [cut_words(str(text)) for text in train_data_5['corpus']]
sentences_as_texts_2 = [' '.join(sentence) for sentence in sentences]


# 构建CountVectorizer对象，并进行向量化
vectorizer = CountVectorizer()

ref_vecs = vectorizer.fit_transform(sentences_as_texts_1)
dom_vecs = vectorizer.fit_transform(sentences_as_texts_2)

In [61]:
tf_words = vectorizer.get_feature_names()

/root/miniconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [64]:
def get_log_likelihood(dom_vecs, ref_vecs, tf_words, n_words):  
    
    # get word occurances in domain corpus
    O_dom = np.array(dom_vecs.sum(axis = 0).tolist()[0])
    O_dom = O_dom.astype(float)
    O_dom[O_dom == 0] = 0.00001  # avoid division by 0
    
    # get word occurances in ref corpus
    O_ref = np.array(ref_vecs.sum(axis = 0).tolist()[0])
    O_ref = O_ref.astype(float)
    O_ref[O_ref == 0] = 0.00001  # avoid division by 0
    
    # get average observed frequencies
    total_corpora_size = dom_vecs.shape[0]+ref_vecs.shape[0]
    aved_freqs = (O_dom+O_ref)/total_corpora_size
    
    # get expected frequencies
    E_dom = dom_vecs.shape[0]*aved_freqs
    E_ref = ref_vecs.shape[0]*aved_freqs
    
    # compute log likelihoods
    LL_ref = O_ref*np.log(O_ref/E_ref)
    LL_dom = O_dom*np.log(O_dom/E_dom)
    LL = 2*(LL_dom + LL_ref)
    
    # sort domain terms 
    sorted_dom_words = [tf_words[i] for i in LL.argsort()[::-1]]
    
    i = 0
    top_dom_words = []
    
    while len(top_dom_words) < n_words:
        
        word = sorted_dom_words[i]
        idx = tf_words.index(word)
        dom_ratio = O_dom[idx]/dom_vecs.shape[0]
        ref_ratio = O_ref[idx]/ref_vecs.shape[0]
    
        if dom_ratio > ref_ratio:
            LL_score = LL[idx]
            top_dom_words.append((word,round(LL_score,2)))
            
        i+=1
        
    return top_dom_words


In [66]:
get_log_likelihood(dom_vecs, ref_vecs, tf_words, 5)

KeyboardInterrupt: 